# Middleware

이 워크샵에서는 Agent Framework에서 Middleware를 사용하여 실행 흐름을 제어하고 확장하는 방법을 학습합니다.

## 목차
1. [Middleware란 무엇인가?](#middleware란-무엇인가)
2. [환경 설정](#환경-설정)
3. [Decorator 기반 Middleware](#decorator-기반-middleware)
4. [Class 기반 Middleware](#class-기반-middleware)
5. [Chat Middleware](#chat-middleware)
6. [Agent-Level vs Run-Level Middleware](#agent-level-vs-run-level-middleware)
7. [Shared State Middleware](#shared-state-middleware)

## 학습 목표
- Middleware의 개념과 작동 방식 이해
- 다양한 Middleware 구현 방법 학습
- Agent, Function, Chat Middleware의 차이점 이해
- 실전 예제를 통한 활용 방법 습득

## 1. Middleware란 무엇인가?

**Middleware**는 Agent 실행의 다양한 단계에서 동작을 가로채고 수정할 수 있는 메커니즘입니다.

### Middleware 유형

1. **Agent Middleware** 🤖
   - Agent 실행 전후에 호출됨
   - 요청/응답 수정 가능
   - 보안 검증, 성능 모니터링 등

2. **Function Middleware** ⚙️
   - Agent 내부의 함수 호출을 가로챔
   - 로깅, 검증, 결과 수정 가능
   - 함수별 세밀한 제어

3. **Chat Middleware** 💬
   - AI 모델로 전송되는 채팅 요청 가로채기
   - 입력 메시지 관찰/수정
   - 응답 전체 오버라이드 가능

### 구현 방식

- **Function-based**: 간단한 async 함수
- **Class-based**: 기본 클래스 상속
- **Decorator-based**: `@agent_middleware`, `@function_middleware` 사용

### 실행 흐름

```
Middleware 1 (before)
  └─> Middleware 2 (before)
        └─> Agent/Function 실행
  ┌─── Middleware 2 (after)
Middleware 1 (after)
```

## 2. 환경 설정

필요한 패키지를 설치하고 환경 변수를 설정합니다.

In [1]:
# 필요한 패키지 설치
# !pip install agent-framework agent-framework-azure-ai python-dotenv

In [2]:
# 환경 변수 로드
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드 (override=True로 기존 값 덮어쓰기)
load_dotenv(override=True)

# 필요한 환경 변수 확인
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_AI_MODEL_DEPLOYMENT_NAME",
]

print("환경 변수 확인:")
for var in required_vars:
    status = "✓" if os.getenv(var) else "✗"
    print(f"{status} {var}")

환경 변수 확인:
✓ AZURE_AI_PROJECT_ENDPOINT
✓ AZURE_AI_MODEL_DEPLOYMENT_NAME


In [3]:
# 공통 import
import datetime
import time
from typing import Annotated
from collections.abc import Awaitable, Callable

from agent_framework import (
    agent_middleware,
    function_middleware,
    chat_middleware,
    AgentMiddleware,
    FunctionMiddleware,
    ChatMiddleware,
    AgentRunContext,
    FunctionInvocationContext,
    ChatContext,
    ChatMessage,
    ChatResponse,
    AgentRunResponse,
    Role,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✓ Import 완료")

✓ Import 완료


## 3. Decorator 기반 Middleware

Decorator를 사용하면 타입 어노테이션 없이도 명시적으로 middleware 유형을 선언할 수 있습니다.

### 3.1 기본 도구 정의

In [4]:
def get_current_time() -> str:
    """현재 시간을 반환합니다."""
    return f"Current time is {datetime.datetime.now().strftime('%H:%M:%S')}"


print("✓ get_current_time 함수 정의 완료")

✓ get_current_time 함수 정의 완료


### 3.2 Decorator를 사용한 Middleware 정의

In [5]:
@agent_middleware  # Agent middleware로 명시적 선언
async def simple_agent_middleware(context, next):  # 타입 어노테이션 불필요
    """Agent 실행 전후에 실행되는 middleware"""
    print("[Agent Middleware] Agent 실행 전")
    await next(context)
    print("[Agent Middleware] Agent 실행 후")


@function_middleware  # Function middleware로 명시적 선언
async def simple_function_middleware(context, next):  # 타입 어노테이션 불필요
    """함수 호출 전후에 실행되는 middleware"""
    print(f"[Function Middleware] 함수 호출 전: {context.function.name}")
    await next(context)
    print(f"[Function Middleware] 함수 호출 후: {context.function.name}")


print("✓ Decorator 기반 middleware 정의 완료")

✓ Decorator 기반 middleware 정의 완료


### 3.3 Decorator Middleware 테스트

In [6]:
async def test_decorator_middleware():
    """Decorator 기반 middleware 테스트"""
    print("=== Decorator Middleware 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="TimeAgent",
            instructions="You are a helpful time assistant. Call get_current_time when asked about time.",
            tools=get_current_time,
            middleware=[simple_agent_middleware, simple_function_middleware],
        ) as agent,
    ):
        query = "What time is it?"
        print(f"\n[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text if result.text else 'No response'}")

await test_decorator_middleware()

=== Decorator Middleware 예제 ===

[사용자] What time is it?
[Agent Middleware] Agent 실행 전


[2025-11-04 08:57:09 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[Function Middleware] 함수 호출 전: get_current_time
[Function Middleware] 함수 호출 후: get_current_time
[Agent Middleware] Agent 실행 후
[Agent] The current time is 08:57:09.


## 4. Class 기반 Middleware

Class를 사용하면 상태를 가진 복잡한 middleware를 구현할 수 있습니다.

### 4.1 날씨 도구 정의

In [7]:
from random import randint
from pydantic import Field


def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """지정된 위치의 날씨를 반환합니다."""
    conditions = ["맑음", "흐림", "비", "폭풍"]
    return f"{location}의 날씨는 {conditions[randint(0, 3)]}이며 최고 기온은 {randint(10, 30)}°C입니다."


print("✓ get_weather 함수 정의 완료")

✓ get_weather 함수 정의 완료


### 4.2 보안 Agent Middleware

In [8]:
class SecurityAgentMiddleware(AgentMiddleware):
    """보안 검증을 수행하는 Agent middleware"""

    async def process(
        self,
        context: AgentRunContext,
        next: Callable[[AgentRunContext], Awaitable[None]],
    ) -> None:
        # 마지막 사용자 메시지에서 보안 위반 검사
        last_message = context.messages[-1] if context.messages else None
        if last_message and last_message.text:
            query = last_message.text
            if "password" in query.lower() or "secret" in query.lower():
                print("[SecurityMiddleware] 🚫 보안 경고: 민감한 정보 감지, 요청 차단.")
                # 경고 메시지로 결과 오버라이드
                context.result = AgentRunResponse(
                    messages=[
                        ChatMessage(
                            role=Role.ASSISTANT,
                            text="민감한 정보가 감지되어 요청이 차단되었습니다."
                        )
                    ]
                )
                # next()를 호출하지 않아 실행 중단
                return

        print("[SecurityMiddleware] ✓ 보안 검사 통과.")
        await next(context)


print("✓ SecurityAgentMiddleware 정의 완료")

✓ SecurityAgentMiddleware 정의 완료


### 4.3 로깅 Function Middleware

In [9]:
class LoggingFunctionMiddleware(FunctionMiddleware):
    """함수 호출을 로깅하는 Function middleware"""

    async def process(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        function_name = context.function.name
        print(f"[LoggingMiddleware] 📝 함수 호출 시작: {function_name}")

        start_time = time.time()
        await next(context)
        end_time = time.time()
        
        duration = end_time - start_time
        print(f"[LoggingMiddleware] ⏱️  함수 {function_name} 완료: {duration:.5f}초")


print("✓ LoggingFunctionMiddleware 정의 완료")

✓ LoggingFunctionMiddleware 정의 완료


### 4.4 Class 기반 Middleware 테스트

In [10]:
async def test_class_based_middleware():
    """Class 기반 middleware 테스트"""
    print("=== Class 기반 Middleware 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant.",
            tools=get_weather,
            middleware=[SecurityAgentMiddleware(), LoggingFunctionMiddleware()],
        ) as agent,
    ):
        # 정상 쿼리 테스트
        print("\n--- 정상 쿼리 ---")
        query = "What's the weather like in Seoul?"
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text}\n")

        # 보안 위반 테스트
        print("--- 보안 테스트 ---")
        query = "What's the password for the weather service?"
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text}\n")

await test_class_based_middleware()

=== Class 기반 Middleware 예제 ===

--- 정상 쿼리 ---
[사용자] What's the weather like in Seoul?
[SecurityMiddleware] ✓ 보안 검사 통과.


[2025-11-04 08:57:19 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[LoggingMiddleware] 📝 함수 호출 시작: get_weather
[LoggingMiddleware] ⏱️  함수 get_weather 완료: 0.00012초
[Agent] The weather in Seoul is cloudy, with a high temperature of 21°C.

--- 보안 테스트 ---
[사용자] What's the password for the weather service?
[SecurityMiddleware] 🚫 보안 경고: 민감한 정보 감지, 요청 차단.
[Agent] 민감한 정보가 감지되어 요청이 차단되었습니다.



## 5. Chat Middleware

Chat Middleware는 AI 모델로 전송되는 채팅 요청을 가로채서 입력을 관찰하거나 수정할 수 있습니다.

### 5.1 입력 관찰 Chat Middleware

In [11]:
class InputObserverMiddleware(ChatMiddleware):
    """입력 메시지를 관찰하고 수정하는 Chat middleware"""

    def __init__(self, replacement: str | None = None):
        """replacement를 지정하면 사용자 메시지를 대체합니다."""
        self.replacement = replacement

    async def process(
        self,
        context: ChatContext,
        next: Callable[[ChatContext], Awaitable[None]],
    ) -> None:
        """AI로 전송되기 전에 입력 메시지를 관찰하고 수정합니다."""
        print("[InputObserver] 📥 입력 메시지 관찰:")

        for i, message in enumerate(context.messages):
            content = message.text if message.text else str(message.contents)
            print(f"  메시지 {i + 1} ({message.role.value}): {content}")

        print(f"[InputObserver] 총 메시지 수: {len(context.messages)}")

        # 사용자 메시지 수정
        if self.replacement:
            modified_messages: list[ChatMessage] = []
            for message in context.messages:
                if message.role == Role.USER and message.text:
                    original_text = message.text
                    print(f"[InputObserver] 🔄 수정: '{original_text}' -> '{self.replacement}'")
                    modified_message = ChatMessage(role=message.role, text=self.replacement)
                    modified_messages.append(modified_message)
                else:
                    modified_messages.append(message)
            
            context.messages[:] = modified_messages

        await next(context)
        print("[InputObserver] ✓ 처리 완료")


print("✓ InputObserverMiddleware 정의 완료")

✓ InputObserverMiddleware 정의 완료


### 5.2 보안 및 오버라이드 Chat Middleware

In [12]:
@chat_middleware
async def security_and_override_middleware(
    context: ChatContext,
    next: Callable[[ChatContext], Awaitable[None]],
) -> None:
    """보안 필터링 및 응답 오버라이드를 구현하는 middleware"""
    print("[SecurityChatMiddleware] 🔍 입력 처리 중...")

    # 보안 검사 - 민감한 정보 차단
    blocked_terms = ["password", "secret", "api_key", "token"]

    for message in context.messages:
        if message.text:
            message_lower = message.text.lower()
            for term in blocked_terms:
                if term in message_lower:
                    print(f"[SecurityChatMiddleware] 🚫 차단됨: '{term}' 감지")

                    # AI 호출 대신 응답 오버라이드
                    context.result = ChatResponse(
                        messages=[
                            ChatMessage(
                                role=Role.ASSISTANT,
                                text="민감한 정보가 포함된 요청은 처리할 수 없습니다. "
                                "비밀번호, 시크릿 등의 정보를 제외하고 다시 질문해주세요.",
                            )
                        ]
                    )

                    # 실행 중단 플래그 설정
                    context.terminate = True
                    return

    # 다음 middleware 또는 AI 실행
    await next(context)


print("✓ security_and_override_middleware 정의 완료")

✓ security_and_override_middleware 정의 완료


### 5.3 Chat Middleware 테스트

In [13]:
async def test_chat_middleware():
    """Chat Middleware 테스트"""
    print("=== Chat Middleware 예제 ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="ChatAgent",
            instructions="You are a helpful AI assistant.",
            middleware=[InputObserverMiddleware(), security_and_override_middleware],
        ) as agent,
    ):
        # 정상 쿼리
        print("\n--- 시나리오 1: 정상 쿼리 ---")
        query = "Hello, how are you?"
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text if result.text else 'No response'}\n")

        # 보안 위반
        print("--- 시나리오 2: 보안 위반 ---")
        query = "What is my password for this account?"
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text if result.text else 'No response'}\n")

await test_chat_middleware()

=== Chat Middleware 예제 ===

--- 시나리오 1: 정상 쿼리 ---
[사용자] Hello, how are you?
[InputObserver] 📥 입력 메시지 관찰:
  메시지 1 (system): You are a helpful AI assistant.
  메시지 2 (user): Hello, how are you?
[InputObserver] 총 메시지 수: 2
[SecurityChatMiddleware] 🔍 입력 처리 중...
[InputObserver] ✓ 처리 완료
[Agent] Hello! I'm just a computer program, so I don't have feelings, but I'm ready to help you with whatever you need. How can I assist you today?

--- 시나리오 2: 보안 위반 ---
[사용자] What is my password for this account?
[InputObserver] 📥 입력 메시지 관찰:
  메시지 1 (system): You are a helpful AI assistant.
  메시지 2 (user): What is my password for this account?
[InputObserver] 총 메시지 수: 2
[SecurityChatMiddleware] 🔍 입력 처리 중...
[SecurityChatMiddleware] 🚫 차단됨: 'password' 감지
[InputObserver] ✓ 처리 완료
[Agent] 민감한 정보가 포함된 요청은 처리할 수 없습니다. 비밀번호, 시크릿 등의 정보를 제외하고 다시 질문해주세요.



## 6. Agent-Level vs Run-Level Middleware

Middleware는 두 가지 레벨에서 적용될 수 있습니다:

- **Agent-level**: Agent 생성 시 지정, 모든 실행에 적용
- **Run-level**: 특정 실행에만 적용

### 6.1 Agent-Level Middleware

In [14]:
async def performance_monitor_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """모든 실행에 대한 성능 모니터링"""
    print("[PerformanceMonitor] ⏱️  성능 모니터링 시작...")
    start_time = time.time()

    await next(context)

    end_time = time.time()
    duration = end_time - start_time
    print(f"[PerformanceMonitor] ⏱️  총 실행 시간: {duration:.3f}초")
    context.metadata["execution_time"] = duration


print("✓ performance_monitor_middleware 정의 완료")

✓ performance_monitor_middleware 정의 완료


### 6.2 Run-Level Middleware

In [15]:
class HighPriorityMiddleware(AgentMiddleware):
    """우선순위가 높은 요청을 위한 Run-level middleware"""

    async def process(
        self,
        context: AgentRunContext,
        next: Callable[[AgentRunContext], Awaitable[None]],
    ) -> None:
        print("[HighPriority] 🚀 높은 우선순위로 처리 중...")

        # Agent-level middleware가 설정한 메타데이터 읽기
        if context.metadata.get("security_validated"):
            print("[HighPriority] ✓ 보안 검증 확인됨")

        # 우선순위 플래그 설정
        context.metadata["priority"] = "high"
        context.metadata["expedited"] = True

        await next(context)
        print("[HighPriority] ✓ 우선순위 처리 완료")


async def debugging_middleware(
    context: AgentRunContext,
    next: Callable[[AgentRunContext], Awaitable[None]],
) -> None:
    """특정 실행의 디버깅을 위한 Run-level middleware"""
    print("[Debug] 🐛 디버그 모드 활성화")
    print(f"[Debug] 메시지 수: {len(context.messages)}")
    print(f"[Debug] 스트리밍 여부: {context.is_streaming}")

    # 기존 메타데이터 로깅
    if context.metadata:
        print(f"[Debug] 기존 메타데이터: {context.metadata}")

    context.metadata["debug_enabled"] = True

    await next(context)

    print("[Debug] ✓ 디버그 정보 수집 완료")


print("✓ Run-level middleware 정의 완료")

✓ Run-level middleware 정의 완료


### 6.3 Agent-Level vs Run-Level 비교 테스트

In [16]:
async def test_agent_vs_run_level():
    """Agent-level과 Run-level middleware 비교"""
    print("=== Agent-Level vs Run-Level Middleware ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant.",
            tools=get_weather,
            # Agent-level middleware: 모든 실행에 적용
            middleware=[
                SecurityAgentMiddleware(),
                performance_monitor_middleware,
            ],
        ) as agent,
    ):
        print("\nAgent 생성 완료 (Agent-level middleware 적용)")
        print("   - SecurityMiddleware")
        print("   - PerformanceMonitor\n")

        # Run 1: Agent-level middleware만 사용
        print("=" * 60)
        print("RUN 1: 정상 쿼리 (Agent-level middleware만)")
        print("=" * 60)
        query = "What's the weather like in Paris?"
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[Agent] {result.text if result.text else 'No response'}\n")

        # Run 2: Agent-level + Run-level middleware
        print("=" * 60)
        print("RUN 2: 우선순위 높은 요청 (Agent + Run-level middleware)")
        print("=" * 60)
        query = "What's the weather in Tokyo? This is urgent!"
        print(f"[사용자] {query}")
        result = await agent.run(
            query,
            middleware=HighPriorityMiddleware(),  # Run-level middleware 추가
        )
        print(f"[Agent] {result.text if result.text else 'No response'}\n")

        # Run 3: 디버그 모드
        print("=" * 60)
        print("RUN 3: 디버그 모드 (Agent + Run-level debugging)")
        print("=" * 60)
        query = "What's the weather in London?"
        print(f"[사용자] {query}")
        result = await agent.run(
            query,
            middleware=[debugging_middleware],  # Run-level debugging
        )
        print(f"[Agent] {result.text if result.text else 'No response'}\n")

await test_agent_vs_run_level()

=== Agent-Level vs Run-Level Middleware ===

Agent 생성 완료 (Agent-level middleware 적용)
   - SecurityMiddleware
   - PerformanceMonitor

RUN 1: 정상 쿼리 (Agent-level middleware만)
[사용자] What's the weather like in Paris?
[SecurityMiddleware] ✓ 보안 검사 통과.
[PerformanceMonitor] ⏱️  성능 모니터링 시작...


[2025-11-04 08:57:31 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[PerformanceMonitor] ⏱️  총 실행 시간: 5.907초
[Agent] The weather in Paris is cloudy with a high temperature of 27°C.

RUN 2: 우선순위 높은 요청 (Agent + Run-level middleware)
[사용자] What's the weather in Tokyo? This is urgent!
[SecurityMiddleware] ✓ 보안 검사 통과.
[PerformanceMonitor] ⏱️  성능 모니터링 시작...
[HighPriority] 🚀 높은 우선순위로 처리 중...


[2025-11-04 08:57:36 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[HighPriority] ✓ 우선순위 처리 완료
[PerformanceMonitor] ⏱️  총 실행 시간: 4.650초
[Agent] The weather in Tokyo is rainy, with a high temperature of 29°C. If you need to go outside, don't forget an umbrella!

RUN 3: 디버그 모드 (Agent + Run-level debugging)
[사용자] What's the weather in London?
[SecurityMiddleware] ✓ 보안 검사 통과.
[PerformanceMonitor] ⏱️  성능 모니터링 시작...
[Debug] 🐛 디버그 모드 활성화
[Debug] 메시지 수: 1
[Debug] 스트리밍 여부: False


[2025-11-04 08:57:40 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[Debug] ✓ 디버그 정보 수집 완료
[PerformanceMonitor] ⏱️  총 실행 시간: 4.533초
[Agent] The weather in London is clear and sunny, with a high temperature of 19°C.



## 7. Shared State Middleware

클래스 내부에 함수 기반 middleware를 정의하여 여러 middleware 간에 상태를 공유할 수 있습니다.

### 7.1 추가 도구 정의

In [17]:
def get_time(
    timezone: Annotated[str, Field(description="The timezone to get the time for.")] = "UTC",
) -> str:
    """지정된 시간대의 현재 시간을 반환합니다."""
    return f"The current time in {timezone} is {datetime.datetime.now().strftime('%H:%M:%S')}"


print("✓ get_time 함수 정의 완료")

✓ get_time 함수 정의 완료


### 7.2 Shared State Middleware Container

In [18]:
class MiddlewareContainer:
    """공유 상태를 가진 middleware 함수들을 담는 컨테이너 클래스"""

    def __init__(self) -> None:
        # 공유 상태: 함수 호출 카운트
        self.call_count: int = 0

    async def call_counter_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """첫 번째 middleware: 공유 상태의 호출 카운트 증가"""
        # 공유 호출 카운트 증가
        self.call_count += 1

        print(f"[CallCounter] 📊 함수 호출 #{self.call_count}")

        # 다음 middleware/함수 호출
        await next(context)

    async def result_enhancer_middleware(
        self,
        context: FunctionInvocationContext,
        next: Callable[[FunctionInvocationContext], Awaitable[None]],
    ) -> None:
        """두 번째 middleware: 공유 호출 카운트를 사용하여 결과 향상"""
        print(f"[ResultEnhancer] 📈 현재까지 총 호출 수: {self.call_count}")

        # 다음 middleware/함수 호출
        await next(context)

        # 함수 실행 후, 공유 상태를 사용하여 결과 향상
        if context.result:
            enhanced_result = f"[호출 #{self.call_count}] {context.result}"
            context.result = enhanced_result
            print("[ResultEnhancer] ✨ 호출 번호로 결과 향상 완료")


print("✓ MiddlewareContainer 정의 완료")

✓ MiddlewareContainer 정의 완료


### 7.3 Shared State Middleware 테스트

In [19]:
async def test_shared_state_middleware():
    """공유 상태를 가진 middleware 테스트"""
    print("=== Shared State Middleware 예제 ===")

    # 공유 상태를 가진 middleware 컨테이너 생성
    middleware_container = MiddlewareContainer()

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="UtilityAgent",
            instructions="You are a helpful assistant that can provide weather information and current time.",
            tools=[get_weather, get_time],
            # 같은 컨테이너 인스턴스의 두 middleware 함수 전달
            # 순서 중요: counter가 먼저 실행되어 카운트 증가,
            # 그 다음 result enhancer가 업데이트된 카운트 사용
            middleware=[
                middleware_container.call_counter_middleware,
                middleware_container.result_enhancer_middleware,
            ],
        ) as agent,
    ):
        # 여러 요청을 테스트하여 공유 상태 확인
        queries = [
            "What's the weather like in New York?",
            "What time is it in London?",
            "What's the weather in Tokyo?",
        ]

        for i, query in enumerate(queries, 1):
            print(f"\n{'='*60}")
            print(f"쿼리 {i}")
            print(f"{'='*60}")
            print(f"[사용자] {query}")
            result = await agent.run(query)
            print(f"[Agent] {result.text if result.text else 'No response'}")

        # 최종 통계 표시
        print("\n" + "=" * 60)
        print("최종 통계")
        print("=" * 60)
        print(f"총 함수 호출 수: {middleware_container.call_count}")

await test_shared_state_middleware()

=== Shared State Middleware 예제 ===

쿼리 1
[사용자] What's the weather like in New York?


[2025-11-04 08:57:47 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[CallCounter] 📊 함수 호출 #1
[ResultEnhancer] 📈 현재까지 총 호출 수: 1
[ResultEnhancer] ✨ 호출 번호로 결과 향상 완료
[Agent] The weather in New York is rainy, with a high temperature of 11°C. If you’re heading out, you might want to bring an umbrella!

쿼리 2
[사용자] What time is it in London?


[2025-11-04 08:57:51 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[CallCounter] 📊 함수 호출 #2
[ResultEnhancer] 📈 현재까지 총 호출 수: 2
[ResultEnhancer] ✨ 호출 번호로 결과 향상 완료
[Agent] The current time in London is 08:57 AM.

쿼리 3
[사용자] What's the weather in Tokyo?


[2025-11-04 08:57:56 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[CallCounter] 📊 함수 호출 #3
[ResultEnhancer] 📈 현재까지 총 호출 수: 3
[ResultEnhancer] ✨ 호출 번호로 결과 향상 완료
[Agent] The weather in Tokyo is cloudy with a high of 19°C. If you need more detailed information like humidity, wind, or forecast, let me know!

최종 통계
총 함수 호출 수: 3


## 요약 및 베스트 프랙티스

### Middleware 핵심 개념

1. **세 가지 유형**
   - Agent Middleware: Agent 실행 전후
   - Function Middleware: 함수 호출 전후
   - Chat Middleware: AI 모델 호출 전후

2. **구현 방식**
   - Function-based: 간단한 async 함수
   - Class-based: 상태를 가진 복잡한 로직
   - Decorator-based: 명시적 타입 선언

3. **적용 레벨**
   - Agent-level: 모든 실행에 적용
   - Run-level: 특정 실행에만 적용

### 베스트 프랙티스

#### 1. Middleware 선택
```python
# ✓ Good: 간단한 로깅은 function-based
@function_middleware
async def simple_logging(context, next):
    print(f"Calling: {context.function.name}")
    await next(context)

# ✓ Good: 복잡한 상태 관리는 class-based
class StatefulMiddleware(FunctionMiddleware):
    def __init__(self):
        self.call_count = 0
    
    async def process(self, context, next):
        self.call_count += 1
        await next(context)
```

#### 2. 실행 순서 이해
```python
# Middleware는 리스트 순서대로 실행됨
middleware=[
    SecurityMiddleware(),      # 1. 먼저 보안 검사
    LoggingMiddleware(),       # 2. 그 다음 로깅
    PerformanceMiddleware(),   # 3. 마지막으로 성능 측정
]
```

#### 3. 적절한 레벨 선택
```python
# ✓ Good: 모든 요청에 적용할 보안은 Agent-level
agent = client.create_agent(
    middleware=SecurityMiddleware(),  # Agent-level
)

# ✓ Good: 특정 요청만 디버깅은 Run-level
result = await agent.run(
    query,
    middleware=DebugMiddleware(),  # Run-level
)
```

#### 4. Context 활용
```python
# ✓ Good: metadata를 사용한 정보 공유
async def middleware1(context, next):
    context.metadata["processed"] = True
    await next(context)

async def middleware2(context, next):
    if context.metadata.get("processed"):
        # middleware1이 처리했음을 알 수 있음
        pass
    await next(context)
```

#### 5. 실행 제어
```python
# ✓ Good: 조건에 따라 실행 중단
async def security_middleware(context, next):
    if contains_sensitive_info(context):
        context.result = create_error_response()
        return  # next()를 호출하지 않아 중단
    
    await next(context)

# ✓ Good: terminate 플래그 사용
async def chat_security_middleware(context, next):
    if is_blocked(context):
        context.result = create_warning_response()
        context.terminate = True
        return
    
    await next(context)
```

### 일반적인 사용 사례

| 사용 사례 | Middleware 유형 | 추천 구현 방식 |
|---------|---------------|-------------|
| 보안 검증 | Agent | Class-based |
| 성능 모니터링 | Agent/Function | Function-based |
| 함수 로깅 | Function | Function-based |
| 결과 수정 | Function | Function-based |
| 입력 필터링 | Chat | Class-based |
| 호출 카운팅 | Function | Class-based (상태 필요) |
| 예외 처리 | Function | Function-based |
| 디버깅 | All | Function-based |

### 다음 단계

1. **커스텀 Middleware 개발**: 프로젝트에 맞는 middleware 구현
2. **모니터링 통합**: 로깅 및 메트릭 시스템 연동
3. **보안 강화**: 고급 보안 검증 로직 추가
4. **성능 최적화**: 캐싱 및 최적화 middleware 구현
5. **에러 핸들링**: 체계적인 예외 처리 시스템 구축